### First we load libraries, define our train/test split, and load the word2vec dictionary using gensim

In [1]:
from __future__ import print_function
import os
import csv
import random
import gensim
import numpy as np
import pandas as pd
from sklearn.neural_network import MLPClassifier

In [50]:
num_train = 8000
num_dev = 2000
num_test = 2000
num_predict = 20
split_idx = list(range(num_train + num_dev))
random.shuffle(split_idx)

In [3]:
word2vec = gensim.models.KeyedVectors.load_word2vec_format("GoogleNews-vectors-negative300.bin.gz", binary=True)
print("Loaded word vectors successfully!")

Loaded word vectors successfully!



### Parse Descriptions


In [11]:
def parse_descriptions(data_dir, num_doc):
    docs = []
    for i in range(num_doc):
        path = os.path.join(data_dir, "%d.txt" % i)
        with open(path) as f:
            docs.append(f.read())
    return docs

def doc_to_vec(sentence, word2vec):
    # get list of word vectors in sentence
    word_vecs = [word2vec.get_vector(w) for w in sentence.split() if w in word2vec.vocab]
    # return average
    return np.stack(word_vecs).mean(0)

# build x matrices
train_dev_desc = parse_descriptions("data/descriptions_train", num_doc=(num_train+num_dev))
test_desc = parse_descriptions("data/descriptions_test", num_doc=num_test)
d_train = np.array([doc_to_vec(train_dev_desc[i], word2vec) for i in split_idx[:num_train]])
d_dev = np.array([doc_to_vec(train_dev_desc[i], word2vec) for i in split_idx[num_train:]])
d_test = np.array([doc_to_vec(d, word2vec) for d in test_desc])

print("Built all d matrices!")
print("d_train shape:", d_train.shape)
print("d_dev shape:", d_dev.shape)
print("d_test shape:", d_test.shape)

Built all d matrices!
d_train shape: (8000, 300)
d_dev shape: (2000, 300)
d_test shape: (2000, 300)


### Parse Tags

In [9]:
# import bag_of_words.BagofWords
from bag_of_words import BagofWords

In [10]:
def parse_tags(data_dir, num_doc):
    tags = []
    for i in range(num_doc):
        with open(data_dir + str(i) + '.txt') as f:
            lines = f.readlines()
            tag = []
            for line in lines:
                tag.append(line.split(':')[1].rstrip())
            tags.append(tag)
    return tags

train_dev_tags = np.asarray(parse_tags('data/tags_train/', num_doc=(num_train+num_dev)))
test_tags = np.asarray(parse_tags('data/tags_test/', num_doc=(num_test)))
t_train = train_dev_tags[split_idx[:num_train]]
t_dev = train_dev_tags[split_idx[num_train:]]
t_test = test_tags

bow = BagofWords(t_train)
t_train = bow.getFeatures(t_train)
t_dev = bow.getFeatures(t_dev)
t_test = bow.getFeatures(t_test)

print("Built all t matrices!")
print("t_train shape:", t_train.shape)
print("t_dev shape:", t_dev.shape)
print("t_test shape:", t_test.shape)

Built all t matrices!
t_train shape: (8000, 80)
t_dev shape: (2000, 80)
t_test shape: (2000, 80)



### Parse ResNet


In [15]:
def parse_features(features_path):
    vec_map = {}
    with open(features_path) as f:
        for row in csv.reader(f):
            img_id = int(row[0].split("/")[1].split(".")[0])
            vec_map[img_id] = np.array([float(x) for x in row[1:]])
    return np.array([v for k, v in sorted(vec_map.items())])

i_train_dev = parse_features("data/features_train/features_resnet1000_train.csv")
i_train = i_train_dev[split_idx[:num_train]]
i_dev = i_train_dev[split_idx[num_train:]]
i_test = parse_features("data/features_test/features_resnet1000_test.csv") # @ is matrix multiplication for Python 3

print("Built all y matrices!")
print("y_train shape:", i_train.shape)
print("y_dev shape:", i_dev.shape)
print("y_test shape:", i_test.shape)

Built all y matrices!
y_train shape: (8000, 1000)
y_dev shape: (2000, 1000)
y_test shape: (2000, 1000)


In [52]:
from sklearn.decomposition import PCA
pca = PCA(n_components=300)
pca.fit(i_train)
i_train = pca.transform(i_train)
i_dev = pca.transform(i_dev)
i_test = pca.transform(i_test)

### Neural Networks

In [53]:
#Positive Examples
x_train_pos = np.concatenate((d_train, i_train, t_train), axis=1)

#Negative Examples
x_train_neg = [0] * num_train
for i in range(num_train):
    j = random.randint(0,num_train-1)
    while i == j:
        j = random.randint(0,num_train-1)
    x_train_neg[i] = np.concatenate((d_train[i], i_train[j], t_train[j]))
x_train_neg = np.asarray(x_train_neg)

# X data for Neural Network
x_train = np.concatenate((x_train_pos, x_train_neg), axis=0)
print(x_train.shape)

(16000, 680)


In [54]:
#Lables for Neural Network
y_train_pos = np.ones(num_train)
y_train_neg = np.zeros(num_train)
y_train = np.concatenate((y_train_pos, y_train_neg))
print(y_train.shape)

(16000,)


In [55]:
clf = MLPClassifier(hidden_layer_sizes=(680, 680, 680) , activation='relu', solver='adam', learning_rate='adaptive', learning_rate_init=0.01, 
                    max_iter=200, random_state=1, early_stopping=False, validation_fraction=0.1, warm_start=False, verbose=True)
clf.fit(x_train, y_train)

Iteration 1, loss = 1.20021577
Iteration 2, loss = 0.69555866
Iteration 3, loss = 0.69448791
Iteration 4, loss = 0.69372181
Iteration 5, loss = 0.69187643
Iteration 6, loss = 0.68332317
Iteration 7, loss = 0.66601162
Iteration 8, loss = 0.62494358
Iteration 9, loss = 0.56000786
Iteration 10, loss = 0.50581570
Iteration 11, loss = 0.46917107
Iteration 12, loss = 0.44224239
Iteration 13, loss = 0.41346863
Iteration 14, loss = 0.37415275
Iteration 15, loss = 0.34849208
Iteration 16, loss = 0.32918319
Iteration 17, loss = 0.30273855
Iteration 18, loss = 0.28285980
Iteration 19, loss = 0.26715746
Iteration 20, loss = 0.23829419
Iteration 21, loss = 0.23855665
Iteration 22, loss = 0.22841834
Iteration 23, loss = 0.21956649
Iteration 24, loss = 0.20600087
Iteration 25, loss = 0.20736671
Iteration 26, loss = 0.20029406
Iteration 27, loss = 0.18620050
Iteration 28, loss = 0.17655849
Iteration 29, loss = 0.18394252
Iteration 30, loss = 0.17158515
Iteration 31, loss = 0.16394822
Iteration 32, los

MLPClassifier(activation='relu', alpha=0.0001, batch_size='auto', beta_1=0.9,
       beta_2=0.999, early_stopping=False, epsilon=1e-08,
       hidden_layer_sizes=(680, 680, 680), learning_rate='adaptive',
       learning_rate_init=0.01, max_iter=200, momentum=0.9,
       n_iter_no_change=10, nesterovs_momentum=True, power_t=0.5,
       random_state=1, shuffle=True, solver='adam', tol=0.0001,
       validation_fraction=0.1, verbose=True, warm_start=False)

In [56]:
# clf_1300_650 = clf
# clf_1300_1300_650 = clf
clf.n_iter_
clf.n_layers_
clf.loss_

0.13058971785945805

### Prediction

In [59]:
y_dev_score = [0] * num_dev
for i in range(num_dev):
    if i%25 == 0: print(i, end=' ')
    d_dev_rep = np.repeat([d_dev[i]], num_dev, axis=0)
    x_dev = np.concatenate((d_dev_rep, i_dev, t_dev), axis=1)
    score = clf.predict_proba(x_dev)
    y_dev_score[i] = np.transpose(score[:, 1])

0 25 50 75 100 125 150 175 200 225 250 275 300 325 350 375 400 425 450 475 500 525 550 575 600 625 650 675 700 725 750 775 800 825 850 875 900 925 950 975 1000 1025 1050 1075 1100 1125 1150 1175 1200 1225 1250 1275 1300 1325 1350 1375 1400 1425 1450 1475 1500 1525 1550 1575 1600 1625 1650 1675 1700 1725 1750 1775 1800 1825 1850 1875 1900 1925 1950 1975 

In [57]:
i = 1
print(d_dev[i].shape)
d_dev_rep = np.repeat([d_dev[i]], num_dev, axis=0)
print(d_dev_rep.shape)
print(i_dev.shape)
print(t_dev.shape)
x_dev = np.concatenate((d_dev_rep, i_dev, t_dev), axis=1)
print(x_dev.shape)
print(x_dev)
np.sum(x_dev[0]-x_dev[1])

(300,)
(2000, 300)
(2000, 300)
(2000, 80)
(2000, 680)
[[0.01421441 0.05601501 0.03154161 ... 0.         0.         0.        ]
 [0.01421441 0.05601501 0.03154161 ... 0.         0.         0.        ]
 [0.01421441 0.05601501 0.03154161 ... 0.         0.         0.        ]
 ...
 [0.01421441 0.05601501 0.03154161 ... 0.         0.         0.        ]
 [0.01421441 0.05601501 0.03154161 ... 0.         0.         0.        ]
 [0.01421441 0.05601501 0.03154161 ... 0.         0.         0.        ]]


-4.448195191532257

In [58]:
score = clf.predict_proba(x_dev)
print(score)
# score_single = clf.predict_proba(np.concatenate((d_dev[0], i_dev[1], t_dev[1])) )
print( clf.predict_proba([np.concatenate((d_dev[500], i_dev[3], t_dev[1])) ] ) )

[[9.99999249e-01 7.50954586e-07]
 [3.30123125e-03 9.96698769e-01]
 [9.99999963e-01 3.69405185e-08]
 ...
 [9.99817515e-01 1.82485207e-04]
 [9.99999700e-01 3.00292245e-07]
 [9.99997055e-01 2.94453821e-06]]
[[9.99407368e-01 5.92631848e-04]]


### Score Calculation

In [61]:
y_dev_score = np.asarray(y_dev_score)
y_dev_predict = np.argsort(y_dev_score, axis=1)[:, ::-1]

dev_scores = []
dev_pos_list = []
for i in range(num_dev):
    dev_pos = np.where(y_dev_predict[i] == i)
    dev_pos_list.append(dev_pos)
    if dev_pos[0] < 20:
        dev_scores.append((21 - dev_pos[0]) / 20)
    else:
        dev_scores.append(0.0)

print("Development MAP@20:", np.mean(dev_scores, dtype=np.float64))
print("Mean index of true image", np.mean(dev_pos_list))
print("Median index of true image", np.median(dev_pos_list))
print(len(dev_pos_list))

Development MAP@20: 0.01
Mean index of true image 164.6955
Median index of true image 109.0
2000


### Test 

In [94]:
y_test_score = [0] * num_test
for i in range(num_test):
    if i%10 == 0: print(i, end=' ')
    d_test_rep = np.repeat([d_test[i]], num_test, axis=0)
    x_test = np.concatenate((d_test_rep, i_test), axis=1)
    score = clf.predict_proba(x_test)
    y_test_score[i] = np.transpose(score[:, 1])
    
y_test_score = np.asarray(y_test_score)
y_test_predict = np.argsort(y_test_score, axis=1)[:, ::-1]

0 10 20 30 40 50 60 70 80 90 100 110 120 130 140 150 160 170 180 190 200 210 220 230 240 250 260 270 280 290 300 310 320 330 340 350 360 370 380 390 400 410 420 430 440 450 460 470 480 490 500 510 520 530 540 550 560 570 580 590 600 610 620 630 640 650 660 670 680 690 700 710 720 730 740 750 760 770 780 790 800 810 820 830 840 850 860 870 880 890 900 910 920 930 940 950 960 970 980 990 1000 1010 1020 1030 1040 1050 1060 1070 1080 1090 1100 1110 1120 1130 1140 1150 1160 1170 1180 1190 1200 1210 1220 1230 1240 1250 1260 1270 1280 1290 1300 1310 1320 1330 1340 1350 1360 1370 1380 1390 1400 1410 1420 1430 1440 1450 1460 1470 1480 1490 1500 1510 1520 1530 1540 1550 1560 1570 1580 1590 1600 1610 1620 1630 1640 1650 1660 1670 1680 1690 1700 1710 1720 1730 1740 1750 1760 1770 1780 1790 1800 1810 1820 1830 1840 1850 1860 1870 1880 1890 1900 1910 1920 1930 1940 1950 1960 1970 1980 1990 

In [98]:
# convert prediction to '0.jpg'
y_test_predict = y_test_predict[:, :20]
test_predict_str = [None] * num_test
for i in range(num_test):
    res = ' '.join([str(int(x)) + '.jpg' for x in y_test_predict[i]])
    test_predict_str[i] = res # ' '.join([str(int(x)) + '.jpg' for x in test_predict[i]])
    
# write to csv
df = pd.DataFrame(data=test_predict_str)
df.index = [str(x) + '.txt' for x in range(num_test)]
df.to_csv('./neural_net.csv', mode='w', index=True, index_label='Descritpion_ID', header=['Top_20_Image_IDs'])


### Next we test out our linear model on our development data, computing its MAP@20, and investigating the quality of the rankings


In [55]:
def dist_matrix(x1, x2):
    return ((np.expand_dims(x1, 1) - np.expand_dims(x2, 0)) ** 2).sum(2) ** 0.5

# test performance on development set
y_dev_pred = reg.predict(d_dev)
dev_distances = dist_matrix(y_dev_pred, y_dev)
dev_scores = []
dev_pos_list = []

for i in range(num_dev):
    pred_dist_idx = list(np.argsort(dev_distances[i]))
    dev_pos = pred_dist_idx.index(i)
    dev_pos = np.where(y_dev_pred[i] == i)
    dev_pos_list.append(dev_pos)
    if dev_pos < 20:
        dev_scores.append(1 / (dev_pos + 1)) # think it's wrong
    else:
        dev_scores.append(0.0)

print("Development MAP@20:", np.mean(dev_scores))
print("Mean index of true image", np.mean(dev_pos_list))
print("Median index of true image", np.median(dev_pos_list))

('Development MAP@20:', 0.0)
('Mean index of true image', nan)
('Median index of true image', nan)


/Users/Keith/anaconda2/lib/python2.7/site-packages/numpy/core/fromnumeric.py:2920: RuntimeWarning: Mean of empty slice.
  out=out, **kwargs)
/Users/Keith/anaconda2/lib/python2.7/site-packages/numpy/core/_methods.py:85: RuntimeWarning: invalid value encountered in double_scalars
  ret = ret.dtype.type(ret / rcount)


In [ ]:

('Development MAP@20:', 0.042)
('Mean index of true image', 114.067)
('Median index of true image', 31.0)

In [52]:
y_dev_pred.shape
dev_distances.shape

(2000, 2000)


### Finally we use our model to compute top-20 predictions on the test data that can be submitted to Kaggle


In [9]:
# create test predictions
x_train_all = np.concatenate([x_train, x_dev])
y_train_all = np.concatenate([y_train, y_dev])
reg_best.fit(x_train_all, y_train_all)
y_test_pred = reg_best.predict(x_test)
test_distances = dist_matrix(y_test_pred, y_test)
pred_rows = []

for i in range(num_test):
    test_dist_idx = list(np.argsort(test_distances[i]))
    top_20 = test_dist_idx[:20]
    row = ["%d.jpg" % i for i in test_dist_idx[:20]]
    pred_rows.append(" ".join(row))

with open("test_submission.csv", "w") as f:
    f.write("Descritpion_ID,Top_20_Image_IDs\n")
    for i, row in enumerate(pred_rows):
        f.write("%d.txt,%s\n" % (i, row))

print("Output written!")

Output written!
